In [1]:

import pandas as pd
import json

In [2]:
sanders = pd.read_csv("sanders_final.csv")

In [3]:
harris = pd.read_csv("harris_final.csv")

In [4]:
warren = pd.read_csv("Warren_final.csv")

In [5]:
biden = pd.read_csv("Biden_final.csv")

In [6]:
beto = pd.read_csv("beto_final.csv")

In [7]:
df = pd.concat([sanders, warren, harris, biden, beto], ignore_index=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
df["candidate"].describe()

count        403
unique         5
top       Warren
freq         151
Name: candidate, dtype: object

In [9]:
#df = df.dropna()

In [10]:
df['properties.color'].unique()

array(['#f67e7d', '#fbdaee', '#c0e7f6', '#61c9bf', '#FFFACD'],
      dtype=object)

In [11]:
def Convert(string): 
    li = list(map(float, string[1:-1].split(','))) 
    return li

In [12]:
df["geometry.coordinates"] = df["geometry.coordinates"].apply(lambda x: Convert(x))

In [13]:
df["geometry.coordinates"]

0                       [-115.243726, 36.172521]
1               [-71.465246, 42.756471999999995]
2                        [-71.440138, 42.765264]
3      [-115.06454599999999, 36.172996000000005]
4                       [-119.779824, 39.492272]
5                        [-70.991203, 43.262994]
6                [-70.83672800000001, 42.939601]
7                          [-71.30423, 42.80064]
8                                     [0.0, 0.0]
9                        [-91.528757, 41.643749]
10      [-93.57217299999999, 41.672686999999996]
11                       [-93.624483, 41.579341]
12                       [-93.717232, 41.567138]
13                         [-93.06909, 39.21808]
14                       [-93.600254, 42.037879]
15                       [-93.613221, 42.023477]
16                       [-93.719324, 41.613954]
17                         [-92.91631, 41.40805]
18              [-114.97206100000001, 35.927901]
19                      [-115.249544, 36.105967]
20              [-71

In [14]:
df["properties.group_name"] = df["candidate"]

In [15]:
def groups(name):
    if name == 'Sanders':
        return 1
    elif name == 'Warren':
        return 2
    elif name == 'Biden':
        return 3
    elif name == "O'Rourke":
        return 4
    else:
        return 5

In [16]:
df['properties.group_id'] = df['properties.group_name'].apply(lambda x: groups(x))

In [17]:
d_list = list(df["properties.group_name"].unique())

In [18]:
def set_range(x):
    if x in range(0, 10) :
        return 2
    elif x in range(10, 50):
        return 4 
    elif x in range(50, 100):
        return 6
    elif x in range(100, 200):
        return 8
    else:
        return 10

In [19]:
df["properties.radius"] = df.guests.apply(set_range)

In [20]:
ok_json = json.loads(df.to_json(orient='records'))

In [21]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data


In [22]:
geo_format = process_to_geojson(ok_json)

In [23]:
with open('geo-data.js', 'w') as outfile:
    outfile.write("var infoData = ")

with open('geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)
